# Transfer Learning MNIST

* Train a simple convnet on the MNIST dataset the first 5 digits [0..4].
* Freeze convolutional layers and fine-tune dense layers for the classification of digits [5..9].

### Import MNIST data and create 2 datasets with one dataset having digits from 0 to 4 and other from 5 to 9 

#### Import the mnist dataset from keras datasets

In [0]:
%tensorflow_version 1.x
import tensorflow as tf

In [2]:
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from matplotlib import pyplot as plt
import seaborn as sns

Using TensorFlow backend.


#### Creating two datasets one with digits below 5 and one with 5 and above

In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [0]:
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gt5 = x_train[y_train >= 5]
y_train_gt5 = y_train[y_train >= 5] - 5  # make classes start at 0 for
x_test_gt5 = x_test[y_test >= 5]         # np_utils.to_categorical
y_test_gt5 = y_test[y_test >= 5] - 5

### Check 

Verify shapes of x_train, y_train, x_test and y_test for both the datasets with the below given shapes.

In [5]:
print(x_train_lt5.shape)
print(y_train_lt5.shape)
print(x_test_lt5.shape)
print(y_test_lt5.shape)

(30596, 28, 28)
(30596,)
(5139, 28, 28)
(5139,)


In [6]:
print(x_train_gt5.shape)
print(y_train_gt5.shape)
print(x_test_gt5.shape)
print(y_test_gt5.shape)

(29404, 28, 28)
(29404,)
(4861, 28, 28)
(4861,)


### Let us take only the dataset (x_train, y_train, x_test, y_test) for Integers 0 to 4 in MNIST
### Reshape x_train and x_test to a 4 Dimensional array (channel = 1) to pass it into a Conv2D layer

In [0]:
X_train = x_train_lt5.reshape(x_train_lt5.shape[0], 28, 28, 1).astype('float32')
X_test = x_test_lt5.reshape(x_test_lt5.shape[0], 28, 28, 1).astype('float32')

### Change into float32 datatype and Normalize x_train and x_test by dividing it by 255.0

In [0]:
X_train/= 255
X_test/= 255

### Check

Verify the shapes of the X_train and X_test with the shapes given below.

In [9]:
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

X_train shape: (30596, 28, 28, 1)
X_test shape: (5139, 28, 28, 1)


### Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
y_train = np_utils.to_categorical(y_train_lt5, 5)
y_test = np_utils.to_categorical(y_test_lt5, 5)

In [11]:
print('y_train shape: ', y_train.shape)
print('y_test shape: ', y_test.shape)

y_train shape:  (30596, 5)
y_test shape:  (5139, 5)


### Build a sequential model with 2 Convolutional layers with 32 kernels of size (3,3) followed by a Max pooling layer of size (2,2) followed by a drop out layer to be trained for classification of digits 0-4  

In [12]:
model = Sequential()

model.add(Convolution2D(32,3,3,input_shape = (28,28,1)))
model.add(Activation('relu'))

model.add(Convolution2D(32,3,3))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(0.1))






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(28, 28, 1...)`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  


### Post that flatten the data and add 2 Dense layers with 128 neurons and neurons = output classes with activation = 'relu' and 'softmax' respectively. Add dropout layer inbetween if necessary  

In [13]:
model.add(Flatten())

model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dense(5))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=7, verbose=1, mode='auto')
callback_list = [early_stopping]

### Print the training and test accuracy for 5 epochs

In [14]:
epoch = 5
batch = 32

model.fit(X_train, y_train, batch_size=batch, nb_epoch=epoch, validation_data=(X_test, y_test), callbacks=callback_list)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  after removing the cwd from sys.path.




Train on 30596 samples, validate on 5139 samples
Epoch 1/5





30596/30596 [==============================] - 11s 364us/step - loss: 0.0612 - acc: 0.9811 - val_loss: 0.0148 - val_acc: 0.9938
Epoch 2/5
30596/30596 [==============================] - 9s 302us/step - loss: 0.0172 - acc: 0.9941 - val_loss: 0.0079 - val_acc: 0.9975
Epoch 3/5
30596/30596 [==============================] - 9s 303us/step - loss: 0.0096 - acc: 0.9970 - val_loss: 0.0107 - val_acc: 0.9965
Epoch 4/5
30596/30596 [==============================] - 9s 302us/step - loss: 0.0074 - acc: 0.9975 - val_loss: 0.0100 - val_acc: 0.9963
Epoch 5/5
30596/30596 [==============================] - 9s 301us/step - loss: 0.0055 - acc: 0.9978 - val_loss: 0.0072 - val_acc: 0.9979


In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
model.save('/content/drive/My Drive/Colab Notebooks/nn_mnist.h5')
model.save_weights('/content/drive/My Drive/Colab Notebooks/nn_mnist_weights.h5')

### Use the model trained on 0 to 4 digit classification and train it on the dataset which has digits 5 to 9  (Using Transfer learning keeping only the dense layers to be trainable)

### Make only the dense layers to be trainable and convolutional layers to be non-trainable

#### Check model summary to see model layer names

In [17]:
model2 = Sequential()

model2.add(Convolution2D(32,3,3,input_shape = (28,28,1)))
model2.add(Activation('relu'))

model2.add(Convolution2D(32,3,3))
model2.add(Activation('relu'))

model2.add(MaxPooling2D(pool_size=(2,2)))

model2.add(Dropout(0.1))

model2.add(Flatten())

model2.add(Dense(128))
model2.add(Activation('relu'))

model2.add(Dense(5))
model2.add(Activation('softmax'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(28, 28, 1...)`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  


In [18]:
for layers in model2.layers:
    print(layers.name)
    if('dense' not in layers.name):
        layers.trainable = False
        print(layers.name + 'is not trainable\n')
    if('dense' in layers.name):
        print(layers.name + ' is trainable\n')

conv2d_3
conv2d_3is not trainable

activation_5
activation_5is not trainable

conv2d_4
conv2d_4is not trainable

activation_6
activation_6is not trainable

max_pooling2d_2
max_pooling2d_2is not trainable

dropout_2
dropout_2is not trainable

flatten_2
flatten_2is not trainable

dense_3
dense_3 is trainable

activation_7
activation_7is not trainable

dense_4
dense_4 is trainable

activation_8
activation_8is not trainable



In [0]:
model2.load_weights('/content/drive/My Drive/Colab Notebooks/nn_mnist_weights.h5')

model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

### Do the required preprocessing for `x_train_gt5` also same as `x_train_lt5` and for `y_train_gt5` same as `y_train_lt5`

1. Reshape
2. Change to float32 datatype
3. Normalize (dividing with 255)
4. y_train and y_test Convert into one-hot vectors

In [0]:
X_train_gt5 = x_train_gt5.reshape(x_train_gt5.shape[0], 28, 28, 1).astype('float32')
X_test_gt5 = x_test_gt5.reshape(x_test_gt5.shape[0], 28, 28, 1).astype('float32')

X_train_gt5 /= 255
X_test_gt5 /= 255

y_train_gt5 = np_utils.to_categorical(y_train_gt5, 5)
y_test_gt5 = np_utils.to_categorical(y_test_gt5, 5)

### Check

Verify the shapes with the given below.

In [21]:
print(x_train_gt5.shape)
print(y_train_gt5.shape)
print(x_test_gt5.shape)
print(y_test_gt5.shape)

(29404, 28, 28)
(29404, 5)
(4861, 28, 28)
(4861, 5)


## Print the accuracy for classification of digits 5 to 9

In [22]:
model2.fit(X_train_gt5, y_train_gt5, batch_size=batch, nb_epoch=epoch, validation_data=(X_test_gt5, y_test_gt5), callbacks=callback_list)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Train on 29404 samples, validate on 4861 samples
Epoch 1/5
29404/29404 [==============================] - 6s 218us/step - loss: 0.1726 - acc: 0.9580 - val_loss: 0.0322 - val_acc: 0.9887
Epoch 2/5
29404/29404 [==============================] - 6s 205us/step - loss: 0.0252 - acc: 0.9925 - val_loss: 0.0297 - val_acc: 0.9893
Epoch 3/5
29404/29404 [==============================] - 6s 204us/step - loss: 0.0138 - acc: 0.9954 - val_loss: 0.0254 - val_acc: 0.9918
Epoch 4/5
29404/29404 [==============================] - 6s 206us/step - loss: 0.0076 - acc: 0.9982 - val_loss: 0.0179 - val_acc: 0.9938
Epoch 5/5
29404/29404 [==============================] - 6s 205us/step - loss: 0.0054 - acc: 0.9983 - val_loss: 0.0239 - val_acc: 0.9926


In [26]:
loss, accuracy = model2.evaluate(X_test_gt5, y_test_gt5)
print(accuracy)

4861/4861 [==============================] - 0s 88us/step
0.9925941164369472


# Text classification using TF-IDF

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

###  Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### Training data

In [30]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [47]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [48]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [49]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [50]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), min_df=2)
#Vectorizing train 
X_train_dtm = vect.fit_transform(twenty_train.data)
print('Features: ', X_train_dtm.shape[1])    
#Vectorizing test
X_test_dtm = vect.transform(twenty_test.data)
print('Features: ', X_test_dtm.shape[1]) 

Features:  65243
Features:  65243


### Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [51]:
 #Logistic Regression
logreg = LogisticRegression()
logreg.fit(X_train_dtm, twenty_train.target)

y_pred_class = logreg.predict(X_test_dtm)
print('\nTest Accuracy Logistic Regression: ', metrics.accuracy_score(twenty_test.target, y_pred_class))

y_pred_train = logreg.predict(X_train_dtm)
print('Train Accuracy Logistic Regression: ', metrics.accuracy_score(twenty_train.target, y_pred_train))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)



Test Accuracy Logistic Regression:  0.9007989347536618
Train Accuracy Logistic Regression:  0.9937970757642889
